#### This notebook looks at temperature-dependent changes to embryo morphology

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
from glob2 import glob

In [ ]:
# load embryo_df for our current best model
# root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"

root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
train_name = "20241107_ds"
model_name = "SeqVAE_z100_ne150_sweep_01_block01_iter030" 
train_dir = os.path.join(root, "training_data", train_name, "")
output_dir = os.path.join(train_dir, model_name) 

# get path to model
training_path = sorted(glob(os.path.join(output_dir, "*")))[-1]
training_name = os.path.dirname(training_path)
read_path = os.path.join(training_path, "figures", "")

# path to save data
data_path = os.path.join(root, "results", "20240310", "")

# path to figures and data
fig_path = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/slides/morphseq/20250312/morph_metrics_v2/"
os.makedirs(fig_path, exist_ok=True)

### Load more target PCA and spline fits that focus on HF + AB controls

In [ ]:
import joblib

# load datasets
hf_pca_df = pd.read_csv(os.path.join(data_path, "hf_morph_df.csv"))
ref_pca_df = pd.read_csv(os.path.join(data_path, "ab_ref_morph_df.csv"))
spline_df = pd.read_csv(os.path.join(data_path, "spline_morph_df_full.csv"))
spline_df["knot_index"] = spline_df.index

# Save the model to a file
morph_stage_model = joblib.load(os.path.join(data_path, 'morph_stage_model.joblib'))

### Calculate embryo deviations from wt reference

In [ ]:
from scipy.spatial import distance_matrix

pca_cols = [col for col in hf_pca_df.columns if "PCA" in col]
stage_dist_mat = distance_matrix(hf_pca_df[pca_cols], spline_df[pca_cols])

hf_pca_df["knot_index"] = np.argmin(stage_dist_mat, axis=1)
hf_pca_df["dist_to_spline"] = np.min(stage_dist_mat, axis=1)

fig = px.scatter(hf_pca_df, x="knot_index", y="dist_to_spline", color="temperature", 
                 symbol="timepoint", hover_data=["snip_id"])

fig.update_traces(marker=dict(size=8, line=dict(color="black", width=1)))
fig.show()

In [ ]:
hf_pca_df.to_csv(os.path.join(data_path, "hf_morph_dist_df.csv"), index=False)